In [13]:
"""
File v715_optima_simul.py

Michel Bierlaire
Mon Aug 31 14:32:54 2020

"""
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import Beta

# Read the data
df = pd.read_csv('optima.dat', '\t')
database = db.Database('optima', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Define new variables
TimePT_scaled = TimePT / 200
TimeCar_scaled = TimeCar / 200
MarginalCostPT_scaled = MarginalCostPT / 10
CostCarCHF_scaled = CostCarCHF / 10
distance_km_scaled = distance_km / 5
male = Gender == 1
female = Gender == 2
unreportedGender = Gender == -1
fulltime = OccupStat == 1
notfulltime = OccupStat != 1

# Removing some observations
exclude = ((Choice == -1) + ((CarAvail == 3) * (Choice == 1))) != 0
database.remove(exclude)

# List of parameters to be estimated
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 0)
BETA_TIME_FULLTIME = Beta('BETA_TIME_FULLTIME', 0, None, None, 0)
BETA_TIME_OTHER = Beta('BETA_TIME_OTHER', 0, None, None, 0)
BETA_DIST_MALE = Beta('BETA_DIST_MALE', 0, None, None, 0)
BETA_DIST_FEMALE = Beta('BETA_DIST_FEMALE', 0, None, None, 0)
BETA_DIST_UNREPORTED = Beta('BETA_DIST_UNREPORTED', 0, None, None, 0)
BETA_COST = Beta('BETA_COST', 0, None, None, 0)

# Definition of utility functions:
V_PT = BETA_TIME_FULLTIME * TimePT_scaled * fulltime + \
    BETA_TIME_OTHER * TimePT_scaled * notfulltime + \
    BETA_COST * MarginalCostPT_scaled
V_CAR = ASC_CAR + \
    BETA_TIME_FULLTIME * TimeCar_scaled * fulltime + \
    BETA_TIME_OTHER * TimeCar_scaled * notfulltime + \
    BETA_COST * CostCarCHF_scaled
V_SM = ASC_SM + \
    BETA_DIST_MALE * distance_km_scaled * male + \
    BETA_DIST_FEMALE * distance_km_scaled * female + \
    BETA_DIST_UNREPORTED * distance_km_scaled * unreportedGender

# Associate utility functions with the numbering of alternatives
V = {0: V_PT,
     1: V_CAR,
     2: V_SM}

try:
    m = 'v715_optima_model'
    results = res.bioResults(pickleFile=f'{m}.pickle')
except FileNotFoundError:
    print(f'Run first the script {m}.py in order to generate the file '
          '{m}.pickle.')
    sys.exit()

prob_pt = models.logit(V, None, 0)
prob_car = models.logit(V, None, 1)
prob_sm = models.logit(V, None, 2)

simulate = {'Prob. public transp.': prob_pt,
            'Prob. car': prob_car,
            'Prob. slow modes': prob_sm,
            'Weight': Weight}

biosim = bio.BIOGEME(database, simulate)
biosim.modelName = 'v715_optima_simul'

# Perform the simulation
simresults = biosim.simulate(results.getBetaValues())

totalWeight = simresults['Weight'].sum()

simresults['Weighted public'] = simresults['Prob. public transp.'] * simresults['Weight']
simresults['Weighted car'] = simresults['Prob. car'] * simresults['Weight']
simresults['Weighted slow'] = simresults['Prob. slow modes'] * simresults['Weight']


ms_pt = simresults['Weighted public'].sum() / totalWeight
ms_car = simresults['Weighted car'].sum() / totalWeight
ms_sm = simresults['Weighted slow'].sum() / totalWeight
print(f'Market shares for public transportation: {100*ms_pt:.2f}%')
print(f'Market shares for car: {100*ms_car:.2f}%')
print(f'Market shares for slow modes: {100*ms_sm:.2f}%')

Market shares for public transportation: 28.70%
Market shares for car: 65.22%
Market shares for slow modes: 6.08%


In [14]:
totalWeight

0.8044510140000001

In [4]:
simresults['Weight']

0       0.000379
2       0.000368
3       0.000368
4       0.000409
5       0.000368
          ...   
2259    0.000870
2261    0.000368
2262    0.000368
2263    0.000409
2264    0.000409
Name: Weight, Length: 1899, dtype: float64

In [5]:
simresults['Prob. public transp.']

0       0.477677
2       0.276121
3       0.110337
4       0.075302
5       0.253484
          ...   
2259    0.281685
2261    0.138736
2262    0.224006
2263    0.222956
2264    0.210227
Name: Prob. public transp., Length: 1899, dtype: float64

In [6]:
simresults['Weighted public']

0       0.000181
2       0.000102
3       0.000041
4       0.000031
5       0.000093
          ...   
2259    0.000245
2261    0.000051
2262    0.000082
2263    0.000091
2264    0.000086
Name: Weighted public, Length: 1899, dtype: float64

In [7]:
simresults['Weighted public'].sum()

0.23089832631104662

In [8]:
simresults['Weighted public'].sum() / totalWeight

0.287025962168837